In [7]:
print("Please check to create the DB with Beev Electic Vehicle Specs Data")
print("Alex, please answer")

Please check to create the DB with Beev Electic Vehicle Specs Data
Alex, please answer


In [20]:
#LIBRARIES
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#Graphics
import matplotlib.pyplot as plt
import seaborn as sns
#KNeighbors
from sklearn.neighbors import NearestNeighbors
#Scalers
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
#Data selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.decomposition import PCA
#METRICS
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, recall_score,accuracy_score
from sklearn.metrics import mean_squared_error
#RANDOM
import random as rd

In [16]:
df = pd.read_csv("Beev Electric Vehicle Specs Data.csv")

In [17]:
df

,Full Name,Model,Brand,Acceleration,Range,Average Winter Range,Highway Winter Range,City Winter Range,Average Summer Range,Highway Summer Range,...,Type of Driver,Max Speed,Max Trunk Volume (mm),Min Trunk Volume (mm),youtube_video,Creation Date,Modified Date,Slug,Creator,unique id
0,Aiways U5,U5,Aiways,7.5,410.0,285.0,235.0,340.0,385.0,300.0,...,NaN,150.0,1555.0,432.0,NaN,"Sep 19, 2021 20:50","Oct 20, 2021 15:48",NaN,(App admin),1632077402600x431044176816571400
1,Audi e-tron 50 quattro,e-tron 50,Audi,6.8,280.0,240.0,205.0,285.0,320.0,255.0,...,1.0,190.0,1725.0,660.0,NaN,"Jul 20, 2020 15:44","Nov 16, 2021 10:24",NaN,(deleted thing),1595252663946x895911395902033000
2,Audi e-tron 55 quattro,e-tron 55,Audi,5.7,437.0,315.0,265.0,370.0,410.0,330.0,...,1.0,200.0,1725.0,660.0,NaN,"Jul 20, 2020 11:53","Nov 16, 2021 10:29",NaN,(deleted thing),1595238801413x563796210467635700
3,Audi e-tron GT quattro,e-tron GT quattro,Audi,4.1,487.0,355.0,305.0,405.0,475.0,390.0,...,NaN,245.0,NaN,405.0,NaN,"Sep 19, 2021 22:07","Nov 16, 2021 10:37",NaN,(App admin),1632082037901x768751924168235500
4,Audi e-tron GT RS,e-tron GT RS,Audi,3.3,472.0,345.0,295.0,395.0,460.0,375.0,...,NaN,250.0,NaN,366.0,NaN,"Sep 19, 2021 22:13","Nov 16, 2021 10:41",NaN,(App admin),1632082391017x236732476503401120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,Volkswagen ID.5GTX,ID.5 GTX,Volkswagen,6.3,410.0,345.0,296.0,400.0,465.0,375.0,...,NaN,180.0,1561.0,549.0,NaN,"Nov 10, 2021 10:58","Dec 6, 2021 10:21",NaN,(App admin),1636538310300x358194465958965760
146,Volkswagen Transporter 6.1 ABTe,Transporter 6.1 ABTe,Volkswagen,17.4,138.0,NaN,NaN,NaN,NaN,NaN,...,NaN,130.0,NaN,NaN,NaN,"Sep 25, 2021 17:08","Oct 18, 2021 12:51",NaN,(App admin),1632582499541x281936676783606460
147,Volvo XC40 Recharge Twin,XC40 Recharge Twin,Volvo,4.9,400.0,315.0,270.0,365.0,440.0,355.0,...,1.0,180.0,1373.0,413.0,NaN,"Jun 3, 2020 14:40","Nov 15, 2021 19:04",NaN,NaN,1591188039290x449226619630125060
148,Volvo C40 Recharge Twin,C40 Recharge,Volvo,4.7,420.0,290.0,240.0,350.0,380.0,300.0,...,NaN,180.0,1205.0,419.0,NaN,"Sep 19, 2021 23:00","Oct 18, 2021 15:26",NaN,(App admin),1632085219484x265759528215476100


In [31]:
nan_values = df.isna().sum()
nan_values.head(25)

Full Name                                  0
Model                                      0
Brand                                      0
Acceleration                               0
Range                                      0
Average Winter Range                      14
Highway Winter Range                      15
City Winter Range                         15
Average Summer Range                      14
Highway Summer Range                      15
City Summer Range                         16
Optimal Slow Charge                        8
Optimal Fast Charge                       19
Battery Capacity                           0
Useable Battery Capacity                   3
Category                                   0
Government Incentive Category for Help    13
Vehicle Consumption                       26
Average Winter Consumption                19
Highway Winter Consumption                19
City Winter Consumption                   19
Average Summer Consumption                18
Highway Su

In [30]:
df['Acceleration'].fillna(df['Acceleration'].mean(), inplace = True)
df['Range'].fillna(df['Range'].mean(), inplace = True)  
df['Torque'].fillna(df['Torque'].mean(), inplace = True)

In [32]:
X = df[['Acceleration', 'Range', 'Torque']]
distanceKNN = NearestNeighbors(n_neighbors=3).fit(X)

In [47]:
distanceKNN.kneighbors([[10,400,387]], 2, return_distance = False)

C:\Users\Utilizador\anaconda3\envs\Safe space\lib\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


array([[64, 65]], dtype=int64)

In [70]:
df[['Full Name', 'Acceleration' ,'Range','Torque']].iloc[64:66]

,Full Name,Acceleration,Range,Torque
64,MG Marvel R,7.9,402.0,410.0
65,MG Marvel R Luxury,7.0,402.0,410.0


In [55]:
df[df['Acceleration'] == 10]

,Full Name,Model,Brand,Acceleration,Range,Average Winter Range,Highway Winter Range,City Winter Range,Average Summer Range,Highway Summer Range,...,Type of Driver,Max Speed,Max Trunk Volume (mm),Min Trunk Volume (mm),youtube_video,Creation Date,Modified Date,Slug,Creator,unique id
63,Mercedes EQV,EQV,Mercedes,10.0,363.0,285.0,235.0,350.0,370.0,290.0,...,2.0,160.0,4630.0,1030.0,NaN,"Jul 6, 2020 15:37","Oct 20, 2021 20:23",NaN,NaN,1594042637476x963804547928293400
109,Renault Megane E-Tech EV40,Megane E-Tech EV40,Renault,10.0,300.0,205.0,170.0,240.0,280.0,225.0,...,NaN,160.0,NaN,440.0,NaN,"Sep 17, 2021 16:33","Oct 25, 2021 18:33",NaN,(App admin),1631889197168x331708328393404800


In [71]:
df[['Full Name', 'Acceleration', 'Range', 'Torque']][(df['Torque'] > 380) & (df['Torque'] < 390)]

,Full Name,Acceleration,Range,Torque
22,Citroën AMI,8.76831,70.0,387.585034
30,Fiat 500 Berline 24 kWh,9.50000,180.0,387.585034
56,Mercedes EQA 300 4MATIC,7.70000,350.0,387.585034


In [56]:
df[df['Range'] == 400]

,Full Name,Model,Brand,Acceleration,Range,Average Winter Range,Highway Winter Range,City Winter Range,Average Summer Range,Highway Summer Range,...,Type of Driver,Max Speed,Max Trunk Volume (mm),Min Trunk Volume (mm),youtube_video,Creation Date,Modified Date,Slug,Creator,unique id
14,Audi Q4 Sportback e-tron 50 quattro,Q4 Sportback e-tron 50 quattro,Audi,6.2,400.0,340.0,290.0,395.0,460.0,370.0,...,NaN,180.0,1460.0,535.0,NaN,"Sep 19, 2021 21:49","Nov 16, 2021 15:37",NaN,(App admin),1632080960516x403039863361990300
67,MG MG5 Electric,MG5 Electric,MG motor,8.3,400.0,285.0,240.0,335.0,390.0,305.0,...,NaN,180.0,1456.0,578.0,NaN,"Sep 22, 2021 16:31","Oct 25, 2021 18:35",NaN,(App admin),1632321111246x504120110449399550
140,Volkswagen ID.4 GTX,ID.4 GTX,Volkswagen,6.2,400.0,340.0,285.0,395.0,455.0,365.0,...,NaN,180.0,1575.0,543.0,NaN,"Nov 17, 2021 11:46","Nov 17, 2021 11:46",NaN,(App admin),1637145983163x873918697447059600
147,Volvo XC40 Recharge Twin,XC40 Recharge Twin,Volvo,4.9,400.0,315.0,270.0,365.0,440.0,355.0,...,1.0,180.0,1373.0,413.0,NaN,"Jun 3, 2020 14:40","Nov 15, 2021 19:04",NaN,NaN,1591188039290x449226619630125060
149,Volvo XC40 Recharge,XC40 Recharge,Volvo,7.4,400.0,270.0,225.0,325.0,360.0,280.0,...,NaN,160.0,1295.0,419.0,NaN,"Sep 19, 2021 22:49","Nov 15, 2021 19:08",NaN,(App admin),1632084569539x365394090935362370
